# 🧩 8-Puzzle Search Lab - Educational Version
**SE 444: Introduction to AI - Prof. Anis Koubaa**

**Goal**: Understand search algorithms by solving the classic 8-puzzle problem using BFS, DFS, and A* with different heuristics.

**Learning Objectives**:
- Apply the PEAS framework to analyze a search problem
- Understand state space representation
- Compare uninformed vs informed search strategies
- Analyze the impact of different heuristics on search efficiency

---


## 🎯 **Problem Definition: The 8-Puzzle**

The 8-puzzle consists of a 3×3 grid with 8 numbered tiles and one empty space. The goal is to reach a specific target configuration by sliding tiles into the empty space.

### **Example Problem**:
```
Initial State:        Goal State:
┌───┬───┬───┐        ┌───┬───┬───┐
│ 2 │ 8 │ 3 │        │ 1 │ 2 │ 3 │
├───┼───┼───┤        ├───┼───┼───┤
│ 1 │ 6 │ 4 │   →    │ 4 │ 5 │ 6 │
├───┼───┼───┤        ├───┼───┼───┤
│ 7 │   │ 5 │        │ 7 │ 8 │   │
└───┴───┴───┘        └───┴───┴───┘
```

### **Why Study This Problem?**
- Classic AI benchmark problem
- Demonstrates state space search concepts
- Shows trade-offs between different search strategies
- Manageable complexity for educational purposes


## 🔍 **PEAS Analysis**

Let's analyze the 8-puzzle using the **PEAS framework** (Performance, Environment, Actuators, Sensors):

| Component | Description |
|-----------|-------------|
| **Performance** | Find the shortest sequence of moves to reach the goal state |
| **Environment** | 3×3 grid with 8 numbered tiles and 1 empty space |
| **Actuators** | Move tiles up, down, left, or right into the empty space |
| **Sensors** | Can observe the complete current state of the puzzle |

### **Environment Properties**:
- **Fully Observable**: We can see the entire puzzle state
- **Deterministic**: Each action has a predictable outcome
- **Static**: Environment doesn't change during search
- **Discrete**: Finite number of states and actions
- **Known**: We know the rules and goal state


## 🗺️ **State Space Representation**

Understanding how we represent states is crucial for implementing search algorithms.

### **State Representation**:
- **State**: A tuple representing the current configuration, e.g., `(2, 8, 3, 1, 6, 4, 7, 0, 5)`
- **Empty Space**: Represented by 0
- **Goal State**: `(1, 2, 3, 4, 5, 6, 7, 8, 0)`

### **Actions and Transitions**:
From any state, we can move the empty space in up to 4 directions:

```
Current:        After "Up":    After "Left":
┌───┬───┬───┐  ┌───┬───┬───┐  ┌───┬───┬───┐
│ 2 │ 8 │ 3 │  │ 2 │ 8 │ 3 │  │ 2 │ 8 │ 3 │
├───┼───┼───┤  ├───┼───┼───┤  ├───┼───┼───┤
│ 1 │ 6 │ 4 │  │ 1 │   │ 4 │  │   │ 1 │ 4 │
├───┼───┼───┤  ├───┼───┼───┤  ├───┼───┼───┤
│ 7 │   │ 5 │  │ 7 │ 6 │ 5 │  │ 7 │ 6 │ 5 │
└───┴───┴───┘  └───┴───┴───┘  └───┴───┴───┘
```

### **State Space Properties**:
- **Total Possible States**: 9! = 362,880
- **Reachable States**: ~181,440 (only half are solvable)
- **Branching Factor**: 2-4 moves per state (depending on empty space position)
- **Solution Depth**: Varies from 0 to 31 moves for different initial states


## 🔧 **Setup and Environment Check**

Let's start by importing necessary libraries and setting up our environment:


In [1]:
from collections import deque
import heapq
import time

# Check environment
try:
    import google.colab
    print("📍 Running in Google Colab")
except ImportError:
    print("📍 Running in Local Jupyter")

print("✅ Libraries imported successfully!")
print("🎯 Goal State Configuration:")
print("┌───┬───┬───┐")
print("│ 1 │ 2 │ 3 │")
print("├───┼───┼───┤")
print("│ 4 │ 5 │ 6 │")
print("├───┼───┼───┤")
print("│ 7 │ 8 │   │")
print("└───┴───┴───┘")


📍 Running in Local Jupyter
✅ Libraries imported successfully!
🎯 Goal State Configuration:
┌───┬───┬───┐
│ 1 │ 2 │ 3 │
├───┼───┼───┤
│ 4 │ 5 │ 6 │
├───┼───┼───┤
│ 7 │ 8 │   │
└───┴───┴───┘


## 🏗️ **Base PuzzleSearcher Class**

This class provides the foundation for our search algorithms. It includes:
- State representation and manipulation
- Neighbor generation (valid moves)
- Heuristic functions for informed search
- Utility functions for path reconstruction and visualization


In [2]:
class PuzzleSearcher:
    def __init__(self):
        self.nodes_explored = 0
        self.goal_state = (1, 2, 3, 4, 5, 6, 7, 8, 0)  # 0 = empty space
        
    def reset_stats(self):
        """Reset statistics for each search."""
        self.nodes_explored = 0
    
    def print_puzzle(self, state):
        """Pretty print puzzle state with nice formatting."""
        puzzle_2d = [list(state[i:i+3]) for i in range(0, 9, 3)]
        print("┌───┬───┬───┐")
        for i, row in enumerate(puzzle_2d):
            print("│", end="")
            for cell in row:
                if cell == 0:
                    print("   │", end="")
                else:
                    print(f" {cell} │", end="")
            print()
            if i < 2:
                print("├───┼───┼───┤")
        print("└───┴───┴───┘")
    
    def get_neighbors(self, state):
        """Generate all valid neighboring states by moving the empty space."""
        neighbors = []
        empty_idx = state.index(0)
        empty_row, empty_col = empty_idx // 3, empty_idx % 3
        
        # Try moving empty space in 4 directions: up, down, left, right
        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            new_row, new_col = empty_row + dr, empty_col + dc
            # Check if new position is within bounds
            if 0 <= new_row < 3 and 0 <= new_col < 3:
                new_idx = new_row * 3 + new_col
                new_state = list(state)
                # Swap empty space with tile
                new_state[empty_idx], new_state[new_idx] = new_state[new_idx], new_state[empty_idx]
                neighbors.append(tuple(new_state))
        return neighbors
    
    def manhattan_distance(self, state):
        """Calculate Manhattan distance heuristic."""
        distance = 0
        for i, tile in enumerate(state):
            if tile != 0:  # Don't count empty space
                current_row, current_col = i // 3, i % 3
                goal_idx = self.goal_state.index(tile)
                goal_row, goal_col = goal_idx // 3, goal_idx % 3
                distance += abs(current_row - goal_row) + abs(current_col - goal_col)
        return distance
    
    def misplaced_tiles(self, state):
        """Count number of misplaced tiles heuristic."""
        count = 0
        for i, tile in enumerate(state):
            if tile != 0 and tile != self.goal_state[i]:
                count += 1
        return count
    
    def reconstruct_path(self, parent, start, goal):
        """Reconstruct the solution path from goal to start."""
        path = []
        current = goal
        while current is not None:
            path.append(current)
            current = parent.get(current)
        path.reverse()
        return path

print("✅ PuzzleSearcher base class defined!")
print("🔧 Ready to implement search algorithms...")


✅ PuzzleSearcher base class defined!
🔧 Ready to implement search algorithms...


## 📊 **Understanding Heuristic Functions**

Before implementing search algorithms, let's understand the heuristic functions we'll use:

### **1. Manhattan Distance Heuristic**
- **Idea**: Sum of distances each tile must move to reach its goal position
- **Calculation**: For each tile, |current_row - goal_row| + |current_col - goal_col|
- **Admissible**: Never overestimates the actual cost
- **Example**: If tile 8 is at position (1,1) but should be at (2,1), distance = |1-2| + |1-1| = 1

### **2. Misplaced Tiles Heuristic**
- **Idea**: Count how many tiles are not in their correct position
- **Simpler**: Less accurate than Manhattan distance
- **Admissible**: Each misplaced tile needs at least 1 move

Let's test these heuristics on a sample puzzle:


In [3]:
# Create puzzle instance and test heuristics
puzzle = PuzzleSearcher()

# Test state: moderately scrambled puzzle
test_state = (2, 8, 3, 1, 6, 4, 7, 0, 5)

print("🧩 Testing Heuristic Functions")
print("="*40)
print("\n📍 Current State:")
puzzle.print_puzzle(test_state)

print("\n🎯 Goal State:")
puzzle.print_puzzle(puzzle.goal_state)

# Calculate heuristics
manhattan_h = puzzle.manhattan_distance(test_state)
misplaced_h = puzzle.misplaced_tiles(test_state)

print(f"\n📏 Manhattan Distance Heuristic: {manhattan_h}")
print(f"🔢 Misplaced Tiles Heuristic: {misplaced_h}")

print(f"\n💡 Interpretation:")
print(f"   - At least {manhattan_h} moves needed (Manhattan estimate)")
print(f"   - At least {misplaced_h} moves needed (Misplaced tiles estimate)")
print(f"   - Manhattan distance is more informative (higher value = better guidance)")


🧩 Testing Heuristic Functions

📍 Current State:
┌───┬───┬───┐
│ 2 │ 8 │ 3 │
├───┼───┼───┤
│ 1 │ 6 │ 4 │
├───┼───┼───┤
│ 7 │   │ 5 │
└───┴───┴───┘

🎯 Goal State:
┌───┬───┬───┐
│ 1 │ 2 │ 3 │
├───┼───┼───┤
│ 4 │ 5 │ 6 │
├───┼───┼───┤
│ 7 │ 8 │   │
└───┴───┴───┘

📏 Manhattan Distance Heuristic: 9
🔢 Misplaced Tiles Heuristic: 6

💡 Interpretation:
   - At least 9 moves needed (Manhattan estimate)
   - At least 6 moves needed (Misplaced tiles estimate)
   - Manhattan distance is more informative (higher value = better guidance)


## 🔄 **Breadth-First Search (BFS)**

**Strategy**: Explore states level by level, guaranteeing the shortest solution

**How it works**:
1. Use a **queue** (FIFO) to store states to explore
2. Always explore the oldest (shallowest) state first
3. Add all neighbors of current state to the end of queue
4. Keep track of visited states to avoid cycles

**Key Properties**:
- **Complete**: Will find a solution if one exists
- **Optimal**: Guarantees shortest path (minimum steps)
- **Time/Space**: O(b^d) where b=branching factor, d=solution depth
- **Best for**: Finding the shortest solution when all moves have equal cost


In [4]:
def bfs(self, initial_state):
    """BFS: Finds the shortest solution by exploring level by level."""
    if initial_state == self.goal_state:
        return [initial_state], {
            'algorithm': 'BFS', 
            'path_length': 0, 
            'nodes_explored': 1, 
            'time_ms': 0
        }
    
    self.reset_stats()
    start_time = time.time()
    
    # Initialize BFS data structures
    queue = deque([initial_state])  # FIFO queue for level-by-level exploration
    visited = {initial_state}       # Set to avoid revisiting states
    parent = {initial_state: None}  # Track parent for path reconstruction
    
    while queue:
        current = queue.popleft()   # Get oldest (shallowest) state
        self.nodes_explored += 1
        
        # Explore all neighbors
        for neighbor in self.get_neighbors(current):
            if neighbor not in visited:
                visited.add(neighbor)
                parent[neighbor] = current
                
                # Check if we found the goal
                if neighbor == self.goal_state:
                    path = self.reconstruct_path(parent, initial_state, neighbor)
                    return path, {
                        'algorithm': 'BFS',
                        'path_length': len(path) - 1,
                        'nodes_explored': self.nodes_explored,
                        'time_ms': (time.time() - start_time) * 1000
                    }
                
                queue.append(neighbor)  # Add to end of queue
    
    # No solution found
    return None, {
        'algorithm': 'BFS', 
        'path_length': -1, 
        'nodes_explored': self.nodes_explored, 
        'time_ms': (time.time() - start_time) * 1000
    }

# Add method to class
PuzzleSearcher.bfs = bfs

print("✅ BFS algorithm implemented!")
print("🔄 Ready to find shortest solutions...")


✅ BFS algorithm implemented!
🔄 Ready to find shortest solutions...
